# Проведение классификации (дискриминации) регионов северо-западного федерального окргуа

## Постановка задачи
### Классифицировать регионы северо-западного по уровню жизни населения.

Количество регионов для классификации $N - 9$. Количество факторов $p - 5$. Что удовлетворяет условия проведения процедуры дискриминантного анализа:

$0 < p < N-2$

В качестве факторов выступают **среднедушевые доходы населения,потребительские расходы на душу населения, демографическая нагрузка, смертность населения, младенческая смертность**. В качестве обучющих выборок даны: $M_1$ - **группа регионов центрального федерального округа** (высокий уровень жизни), $M_2$ - **группа регионов приволжского федерального округа** (средний уровень жизни)

In [20]:
# Здесь подгрузим необходимые библиотеки для проведения процедуры дискриминационного анализа

import numpy as np
import pandas as pd
from IPython.display import HTML

## Исходные данные
Обучающая выборка №1
Регионы Центрального федерального округа

In [21]:
M1 = pd.read_csv('M1.csv')
HTML(M1.to_html(index=False))

Центральный федеральный округ,Среднедушевые доходы населения,Потребительские расходы на душу населения,Демографическая нагрузка,Смертность населения,Младенческая смертность
Белогородская область,32352,26072,788,13.3,2.9
Брянская область,28371,24470,802,14.7,3.6
Владимирская область,25358,20905,831,15.6,5.1
Воронежская область,32022,27904,792,14.1,4.1
Ивановская область,25794,20617,812,15.8,4.9
Калужская область,31394,25060,806,14.6,3.9
Костромская область,25285,20803,858,14.7,7.3
Курская область,29149,23024,825,15.0,4.3
Липецкая область,32479,26858,826,14.3,2.9
Московская область,47201,37972,724,12.0,4.0


Обучающая выборка №2 Регионы приволжского федерального округа

In [22]:
M2 = pd.read_csv('M2.csv')
HTML(M2.to_html(index=False))

Приволжский федеральный округ,Среднедушевые доходы населения,Потребительские расходы на душу населения,Демографическая нагрузка,Смертность населения,Младенческая смертность
Республика Башкортостан,30567,26423,792,12.1,6.1
Республика Марий Эл,20864,15981,820,12.0,4.4
Республика Мордовия,19748,15026,745,13.2,4.3
Республика Татарстан,35707,30107,787,11.0,4.9
Удмуртская Республика,25066,19731,822,11.9,4.4
Чувашская Республика,20162,16463,789,12.4,3.0
Пермский край,30588,25366,807,13.2,4.2
Кировская область,23604,19337,892,14.3,3.2
Нижегородская область,33817,27747,803,14.6,5.0
Оренбургская область,24483,19876,819,13.0,4.4


Целевая выборка. $M_0$ - **Регионы северо-западного федерального округа**

In [23]:
M0 = pd.read_csv('M0.csv')
HTML(M0.to_html(index=False))

Северо-Западный федеральный округ,Среднедушевые доходы населения,Потребительские расходы на душу населения,Демографическая нагрузка,Смертность населения,Младенческая смертность
Республика Карелия,30854,25737,823,14.2,4.7
Республика Коми,35356,24639,745,12.0,5.0
Архангельская область,35693,29187,816,13.0,5.7
Вологодская область,28334,21020,833,14.1,5.9
Калининградская область,28905,23736,742,11.8,5.9
Ленинградская область,32306,25835,734,12.5,2.9
Мурманская область,44237,32510,679,11.4,5.0
Новгородская область,26003,23175,882,16.4,5.2
Псковская область,47169,21786,847,16.8,5.6


## Получение обучающих подмножеств

### Вектора средних значений признаков $\overline{X}^{(1)}$,$\overline{X}^{(2)}$ по обучающим выборкам $М_1$ и $М_2$ 

In [24]:
def calcMean(X):
    mean = np.mean(X, axis = 0)
    return np.reshape(np.array(mean), (-1,1))

In [25]:
M1,M2 = M1.iloc[:,1:].to_numpy(), M2.iloc[:,1:].to_numpy()
    
X1,X2 = calcMean(M1),calcMean(M2)
print(X1)
print(X2)

[[2.95484118e+04]
 [2.41147647e+04]
 [8.13411765e+02]
 [1.48000000e+01]
 [4.30588235e+00]]
[[2.59607857e+04]
 [2.13135000e+04]
 [8.08000000e+02]
 [1.30285714e+01]
 [4.43571429e+00]]


### Ковариационные матрицы $S1,S2$ выборок для $М_1$ и $М_2$
### $S_{ij} = E((M_i-E(M_i))(M_j-E(M_j))$

In [26]:
def getCovMatr(M, vec):
    s = np.zeros((len(vec),len(vec)))
    for row in M:
        s += (np.reshape(row,(-1,1)) - vec).dot((np.reshape(row,(-1,1)) - vec).T)
    return s/len(M)  

In [27]:
S1,S2 = getCovMatr(M1,X1),getCovMatr(M2,X2)
print(S1)
print(S2)

[[ 2.48261986e+07  2.00234222e+07 -1.22543522e+05 -4.24017059e+03
  -2.05219066e+03]
 [ 2.00234222e+07  1.66278803e+07 -1.00489374e+05 -3.48039412e+03
  -1.68595744e+03]
 [-1.22543522e+05 -1.00489374e+05  9.41183391e+02  2.15705882e+01
   1.31152249e+01]
 [-4.24017059e+03 -3.48039412e+03  2.15705882e+01  9.75294118e-01
   3.45294118e-01]
 [-2.05219066e+03 -1.68595744e+03  1.31152249e+01  3.45294118e-01
   1.02761246e+00]]
[[ 2.44561573e+07  2.23928644e+07 -1.55486429e+04 -8.02951020e+02
   1.78169337e+03]
 [ 2.23928644e+07  2.08121532e+07 -1.25981429e+04 -6.88292857e+02
   1.70843214e+03]
 [-1.55486429e+04 -1.25981429e+04  1.01885714e+03  1.06785714e+01
  -5.52142857e+00]
 [-8.02951020e+02 -6.88292857e+02  1.06785714e+01  9.90612245e-01
  -1.08877551e-01]
 [ 1.78169337e+03  1.70843214e+03 -5.52142857e+00 -1.08877551e-01
   5.88010204e-01]]


### Обьединенная ковариационная матрица $\hat{S}$
### $\hat{S} = \frac{1}{n_1+n_2-1}(n_1S^{(1)}+n_2S^{(2)})$


In [28]:
def getOCV(matrs,args):
    constant = 1/(np.sum(args) - len(matrs))
    summ = 0
    for index, row, in enumerate(matrs):
        summ +=args[index]*row
    return constant*summ

In [29]:
ocv = getOCV([S1,S2],[len(M1),len(M2)])
print(ocv)

[[ 2.63597096e+07  2.25482165e+07 -7.93420994e+04 -2.87324877e+03
  -3.42880484e+02]
 [ 2.25482165e+07  1.97946245e+07 -6.49894260e+04 -2.37251034e+03
  -1.63559533e+02]
 [-7.93420994e+04 -6.49894260e+04  1.04359026e+03  1.78000000e+01
   5.02271805e+00]
 [-2.87324877e+03 -2.37251034e+03  1.78000000e+01  1.04995074e+00
   1.49852217e-01]
 [-3.42880484e+02 -1.63559533e+02  5.02271805e+00  1.49852217e-01
   8.86260504e-01]]


### Обратная обьединенная ковариационная матрица $\hat{S}^{-1}$

In [30]:
ocvInv = np.linalg.inv(ocv)
print(ocvInv)

[[ 1.64575073e-06 -1.82249061e-06  6.36214911e-06  2.45866943e-04
   2.22746139e-04]
 [-1.82249061e-06  2.09212620e-06 -4.50304692e-06 -1.45173181e-04
  -2.68924565e-04]
 [ 6.36214911e-06 -4.50304692e-06  1.47314034e-03 -1.71953903e-02
  -3.81091191e-03]
 [ 2.45866943e-04 -1.45173181e-04 -1.71953903e-02  1.60377321e+00
  -1.05389842e-01]
 [ 2.22746139e-04 -2.68924565e-04 -3.81091191e-03 -1.05389842e-01
   1.20430072e+00]]


### Вектор дискриминантных множителей $A$
### $A = \hat{S}^{-1}(\overline{X}^{(1)}-\overline{X}^{(2)})$ 

In [31]:
A = np.dot(ocvInv,np.subtract(X1,X2))
print(A)

[[ 1.24010619e-03]
 [-9.24433880e-04]
 [-1.17825533e-02]
 [ 3.23700539e+00]
 [-3.17870067e-01]]


### Значения дискриминантной функции F для каждой из групп:
### $F_i^{k} = \sum_{j=1}^{p} a_jx_{ij} $

In [32]:
def getFuncValues(A,M):
    vec = []
    for row in M:
        vec.append(np.dot(row,A))
    return np.array(vec)

In [33]:
F1 = getFuncValues(A,M1)
F2 = getFuncValues(A,M2)
print(F1)
print(F2)

[[48.86377199]
 [49.55219507]
 [51.2061676 ]
 [48.9220041 ]
 [52.94793446]
 [52.28942833]
 [47.27918416]
 [52.33132292]
 [51.08392881]
 [52.47366501]
 [50.73004728]
 [51.49754178]
 [52.09998639]
 [51.01880461]
 [55.26295468]
 [53.60896255]
 [51.02669118]]
[[41.37698524]
 [38.8839405 ]
 [43.18270132]
 [41.22516758]
 [40.28097403]
 [39.67288823]
 [46.36807485]
 [46.15764402]
 [52.49594239]
 [43.0200028 ]
 [44.25174222]
 [45.34303845]
 [44.84517795]
 [45.17846518]]


### Средние значения F для каждой из групп:
### $\overline{F} = \frac{\sum_{n=1}^{n_k} F^{k}}{q}$

In [34]:
Fm = np.array([np.mean(F1), np.mean(F2)])
print(Fm)

[51.30556417 43.73448177]


### Константа (общее среднее) дискриминации:

In [35]:
Fms = np.mean(Fm) 
print(Fms)

47.52002297101328


## Распределение (дискриминация) объектов из $M_0$ по обучающим выборкам $M_1$, $M_2$:

In [36]:
names = list(M0[M0.columns[0]])
M0 = M0.iloc[:,1:].to_numpy()
F0 = getFuncValues(A,M0)
print(F0)

[[49.24452763]
 [49.54478038]
 [47.93630595]
 [49.65704448]
 [41.48148265]
 [47.0724715 ]
 [52.1173897 ]
 [51.86447827]
 [80.97664812]]


## Сравнение $F_i^{(0)}$ с $\overline{F}$ 

|------------------| $\overline{F}^{(1)} > \overline{F}^{(2)}$ | $\overline{F}^{(1)} > \overline{F}^{(2)}$ |
|------------------|------------------|------------------|
| $F_i^{(0)} > \overline{F}$ | Высокий уровень жизни   |    Средний уровень жизни  |
| $F_i^{(0)} < \overline{F}$ | Средний уровень жизни    |    Высокий уровень жизни  |

In [37]:
def classificate(needle, Fm):
    res, index, Fms = {}, 0, np.mean(Fm)
    for index, Fi in enumerate(needle):
        if Fm[0] > Fm[1]:
            res[names[index]] = [Fi[0],'Высокий'] if (Fi[0] > Fms) else [Fi[0],'Средний'] 
        else:
            res[names[index]] = [Fi[0],'Средний'] if (Fi[0] > Fms) else [Fi[0],'Высокий']
    return res

In [38]:
result = classificate(F0, Fm)
result = pd.DataFrame.from_dict(result, orient = "index", columns = ['F','Уровень жизни'])
HTML(result.to_html())

,F,Уровень жизни
Республика Карелия,49.244528,Высокий
Республика Коми,49.544780,Высокий
Архангельская область,47.936306,Высокий
Вологодская область,49.657044,Высокий
Калининградская область,41.481483,Средний
Ленинградская область,47.072471,Средний
Мурманская область,52.117390,Высокий
Новгородская область,51.864478,Высокий
Псковская область,80.976648,Высокий
